In [24]:
import pandas as pd
import numpy as np 

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

%run util.ipynb

# multilayer perceptron

### default hyperparams 

In [25]:
X, Y = get_data()

In [26]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
# scaling  TODO note paper did not apply scaling prior 

scaler = StandardScaler()

Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

In [28]:
mlp = MLPClassifier(max_iter=1000, random_state=42)

mlp.fit(Xtrain, Ytrain)

MLPClassifier(max_iter=1000, random_state=42)

In [29]:
Ypred = mlp.predict(Xtest)

accuracy_score(Ytest, Ypred)

0.7708333333333334

In [30]:
print(classification_report(Ytest, Ypred))

              precision    recall  f1-score   support

           0       0.73      0.88      0.80        50
           1       0.83      0.65      0.73        46

    accuracy                           0.77        96
   macro avg       0.78      0.77      0.77        96
weighted avg       0.78      0.77      0.77        96



### hyperparm tuning with gridsearch 

In [31]:
# check existing hyperparams 

mlp.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 1000,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 42,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [32]:
param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (150,), (250,), (500,), (1000,)],
    'activation': ['logistic', 'tanh', 'relu']
}

grid_search_mlp = GridSearchCV(estimator=MLPClassifier(max_iter=10000, random_state=42), param_grid=param_grid_mlp, cv=10, scoring='accuracy', n_jobs=-1)

In [33]:
grid_search_mlp.fit(Xtrain, Ytrain)

GridSearchCV(cv=10, estimator=MLPClassifier(max_iter=10000, random_state=42),
             n_jobs=-1,
             param_grid={'activation': ['logistic', 'tanh', 'relu'],
                         'hidden_layer_sizes': [(50,), (100,), (150,), (250,),
                                                (500,), (1000,)]},
             scoring='accuracy')

In [34]:
grid_search_mlp.best_params_, grid_search_mlp.best_score_

({'activation': 'tanh', 'hidden_layer_sizes': (1000,)},
 np.float64(0.7310276679841896))

In [35]:
bestModel = grid_search_mlp.best_estimator_
Ypred_hyper = bestModel.predict(Xtest)

In [36]:
accuracy_score(Ytest, Ypred_hyper)

0.7916666666666666

In [37]:
print(classification_report(Ytest, Ypred_hyper))

              precision    recall  f1-score   support

           0       0.78      0.84      0.81        50
           1       0.81      0.74      0.77        46

    accuracy                           0.79        96
   macro avg       0.79      0.79      0.79        96
weighted avg       0.79      0.79      0.79        96



### check for overfitting 

In [38]:
# Train accuracy

grid_search_mlp.fit(Xtrain, Ytrain)

YtrainPred = grid_search_mlp.best_estimator_.predict(Xtrain)

accuracy_score(Ytrain, YtrainPred)

0.8340807174887892

In [39]:
# cross validaiton acc 

grid_search_mlp.best_score_

np.float64(0.7310276679841896)